In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
from os import listdir
import pandas as pd
import numpy as np


# Analysis of Covid-19 Developmemt via Worldometer Data

## Utils

In [94]:
def get_day_string():
    day = datetime.now().day
    month = datetime.now().month
    return "{}_{}_2020".format(day,month)
x = get_day_string()
print("Today is {}".format(x))

Today is 28_5_2020


## Read data

In [2]:
listdir("data")

['.ipynb_checkpoints',
 '27_5_2020_worldometer_raw.csv',
 '28_5_2020_worldometer_raw.csv']

In [96]:
df = pd.read_csv("data/{}_worldometer_raw.csv".format(get_day_string()))
df.head()

,country,total_cases,total_deaths,total_recovered,active_cases,seriuos_cases,tot_cases_per_1m,death_per_1m,total_tests,total_tests_per_1m,pop
0,world,5788782,357425,2497593.0,2933764.0,52975.0,743.0,45.9,NaN,NaN,NaN
1,USA,1745803,102107,490130.0,1153566.0,17166.0,5277.0,309.0,15875473.0,47988.0,330822304.0
2,Brazil,414661,25697,166647.0,222317.0,8318.0,1952.0,121.0,871839.0,4104.0,212418030.0
3,Russia,370680,3968,142208.0,224504.0,2300.0,2540.0,27.0,9415992.0,64525.0,145928826.0
4,Spain,283849,27118,196958.0,59773.0,854.0,6071.0,580.0,3556567.0,76071.0,46753147.0


In [98]:
# deal with weird data and small countrys later
#df = df[~df.index.isin(weird_countries)]
sub_df = df[df["total_cases"]>1000]
sub_df = sub_df[~sub_df["pop"].isnull()]
sub_df = sub_df[sub_df["pop"]>10000000]
print(len(sub_df))
sub_df.head()

58


,country,total_cases,total_deaths,total_recovered,active_cases,seriuos_cases,tot_cases_per_1m,death_per_1m,total_tests,total_tests_per_1m,pop
1,USA,1745803,102107,490130.0,1153566.0,17166.0,5277.0,309.0,15875473.0,47988.0,330822304.0
2,Brazil,414661,25697,166647.0,222317.0,8318.0,1952.0,121.0,871839.0,4104.0,212418030.0
3,Russia,370680,3968,142208.0,224504.0,2300.0,2540.0,27.0,9415992.0,64525.0,145928826.0
4,Spain,283849,27118,196958.0,59773.0,854.0,6071.0,580.0,3556567.0,76071.0,46753147.0
5,UK,267240,37460,NaN,NaN,1559.0,3939.0,552.0,3798490.0,55981.0,67852992.0


In [100]:
sub_df["cases_rel_world"] = (sub_df["total_cases"] / world_df["total_cases"].iloc[0]).round(3)
sub_df["cases_rel_pop"] = (sub_df["total_cases"] / sub_df["pop"]).round(3)
sub_df["test_rel_pop"] = (sub_df["total_tests"] / sub_df["pop"]).round(3)
sub_df["death_rel_cases"] = (sub_df["total_deaths"] / sub_df["total_cases"]).round(3)
sub_df.head()

,country,total_cases,total_deaths,total_recovered,active_cases,seriuos_cases,tot_cases_per_1m,death_per_1m,total_tests,total_tests_per_1m,pop,cases_rel_world,cases_rel_pop,test_rel_pop,death_rel_cases
1,USA,1745803,102107,490130.0,1153566.0,17166.0,5277.0,309.0,15875473.0,47988.0,330822304.0,0.302,0.005,0.048,0.058
2,Brazil,414661,25697,166647.0,222317.0,8318.0,1952.0,121.0,871839.0,4104.0,212418030.0,0.072,0.002,0.004,0.062
3,Russia,370680,3968,142208.0,224504.0,2300.0,2540.0,27.0,9415992.0,64525.0,145928826.0,0.064,0.003,0.065,0.011
4,Spain,283849,27118,196958.0,59773.0,854.0,6071.0,580.0,3556567.0,76071.0,46753147.0,0.049,0.006,0.076,0.096
5,UK,267240,37460,NaN,NaN,1559.0,3939.0,552.0,3798490.0,55981.0,67852992.0,0.046,0.004,0.056,0.140


In [102]:
display(sub_df.sort_values("death_rel_cases", ascending=False)["death_rel_cases"].head(10))
display(sub_df.sort_values("cases_rel_pop", ascending=False)["cases_rel_pop"].head(10))
display(sub_df.sort_values("cases_rel_world", ascending=False)["cases_rel_world"].head(10))

19    0.163
7     0.156
6     0.143
5     0.140
21    0.128
25    0.120
17    0.110
4     0.096
24    0.086
13    0.077
Name: death_rel_cases, dtype: float64

4     0.006
1     0.005
19    0.005
12    0.004
5     0.004
6     0.004
15    0.004
25    0.003
21    0.003
28    0.003
Name: cases_rel_pop, dtype: float64

1     0.302
2     0.072
3     0.064
4     0.049
5     0.046
6     0.040
7     0.032
8     0.031
9     0.028
10    0.027
Name: cases_rel_world, dtype: float64